In [1]:
# Load Packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
import math
import pickle


#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs, generate_daily_dates_each_month, \
                                                     next_workday, next_weekday, \
                                                     create_df_wk_days, find_week, date_of_next_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date



In [22]:
#Compute Daily Static  Hedging Errors

###################################
# Common Variables initialisation #
###################################

# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "OTM"

#Product type to hedge: either "CE" or "PE"
prod_type = "PE"
# prod_type = "CE"

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path


# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]


In [23]:

# The function pulls the option value of the trade to be hedge for each day
def generate_dict_opt_value(dict_daily_dates_error_cal, monthly_mkt_data, holidays_list, \
                            prod_type, prod_moneyness, output_path, stock_ident, code_id):
    dict_monthly_option_value = {}
    full_dates_list = []
    full_price_list = []
    full_expiry_list = []

    if (code_id == "0A"):
        df_strikes_hedged = pd.read_csv( output_path + "B" + code_id + "_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_strikes_hedged.csv")
    else:
        df_strikes_hedged = pd.read_csv( output_path + "B" + code_id + "_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_strikes_hedged.csv")

    for each_month in dict_daily_dates_error_cal.keys():
        each_month_data = monthly_mkt_data[get_mthly_df_name_from_expiry(datetime.strptime(each_month, "%d-%b-%Y").date(), prod_type, holidays_list, stock_ident)]     
        strike = df_strikes_hedged[df_strikes_hedged["Date"] == each_month]["Strike"].iloc[0]
    
        df_opt_values = each_month_data[each_month_data["Strike Price"] == strike]
        
        #Remove the following line if we need to do with respect to Close price
        if(prod_type == "CE"):
            df_opt_values['Close'] = df_opt_values.apply(lambda x: max(x['Underlying Value']-x['Strike Price'], 0) if (x['Date'] == x['Expiry']) else \
                                                                    x['Settle Price'], axis=1) 
        else:
            df_opt_values['Close'] = df_opt_values.apply(lambda x: max(x['Strike Price']-x['Underlying Value'], 0) if (x['Date'] == x['Expiry']) else \
                                                                    x['Settle Price'], axis=1) 
            

        df_opt_values.assign(x=pd.to_datetime(df_opt_values['Date'])).sort_values('x').drop('x', 1)
        dates_list = df_opt_values["Date"].tolist()
        full_dates_list = full_dates_list + dates_list
        expiry_list = df_opt_values["Expiry"].tolist()
        full_expiry_list = full_expiry_list + expiry_list
        price_list = df_opt_values["Close"].tolist()
        full_price_list = full_price_list + price_list
    
    dict_monthly_option_value = {"Date": full_dates_list , "Expiry": full_expiry_list, \
                                 "Close Price": full_price_list}
    df_monthly_option_value = pd.DataFrame(dict_monthly_option_value)
    
    return df_monthly_option_value

def combine_weekly_option_df(dict_wkly_expiries_each_month, prod_type, input_path, stock_ident):
    list_of_weekly_df = []
    for each_month in dict_wkly_expiries_each_month.keys():
        no_of_weeks = len(dict_wkly_expiries_each_month[each_month])
        for week in range(0, no_of_weeks):
            if (week < no_of_weeks-1):
                start_date = dict_wkly_expiries_each_month[each_month][week].strftime("%d-%b-%Y") 
                expiry_date = dict_wkly_expiries_each_month[each_month][week+1].strftime("%d-%b-%Y")
            else:
                start_date = dict_wkly_expiries_each_month[each_month][week].strftime("%d-%b-%Y") 
                expiry_date = each_month
            
            file_name = "OPTIDX_" + stock_ident + "_" + prod_type + "_" \
                        + start_date +  "_TO_" \
                        + expiry_date + ".csv" 
            df = pd.read_csv(input_path + file_name)            
            list_of_weekly_df.append(df)
    
    weekly_df_prod = pd.concat(list_of_weekly_df, axis=0) 
    return weekly_df_prod


In [24]:
def find_daily_static_hedging_error(df_wk_day_map, df_ce_weekly, df_pe_weekly,\
                                    df_monthly_option_value,\
                                    prod_type, prod_moneyness, \
                                    holidays_list, output_path, code_id):

    df_ce_map = pd.merge(df_wk_day_map, df_ce_weekly, how = 'left', left_on = ['Day', 'Expiry'], right_on = ['Date', 'Expiry'])
    df_pe_map = pd.merge(df_wk_day_map, df_pe_weekly, how = 'left', left_on = ['Day', 'Expiry'], right_on = ['Date', 'Expiry'])

    df_ce_map = df_ce_map[['Week', 'Day', 'Expiry', 'Month', 'Strike Price', 'Settle Price', 'Underlying Value']]
    df_ce_map['Close'] = df_ce_map.apply(lambda x: max(x['Underlying Value'] - x['Strike Price'], 0) if (x['Day'] == x['Expiry']) else \
                                                                    x['Settle Price'], axis=1) 
    df_pe_map = df_pe_map[['Week', 'Day', 'Expiry', 'Month', 'Strike Price', 'Settle Price', 'Underlying Value']]
    df_pe_map['Close'] = df_pe_map.apply(lambda x: max(x['Strike Price']-x['Underlying Value'], 0) if (x['Day'] == x['Expiry']) else \
                                                                    x['Settle Price'], axis=1) 
    
    df_ce_map = df_ce_map[['Week', 'Day', 'Expiry', 'Month', 'Strike Price', 'Close']]
    df_pe_map = df_pe_map[['Week', 'Day', 'Expiry', 'Month', 'Strike Price', 'Close']]

    if (code_id == "0A"):
        ce_weights = pd.read_csv(output_path + "B" + code_id + "_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv")
        pe_weights = pd.read_csv(output_path + "B" + code_id + "_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv")
        cash_invst = pd.read_csv(output_path + "B" + code_id + "_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv")
    else:    
        ce_weights = pd.read_csv(output_path + "B" + code_id + "_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv")
        pe_weights = pd.read_csv(output_path + "B" + code_id + "_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv")
        cash_invst = pd.read_csv(output_path + "B" + code_id + "_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv")
    
    df_ce_pfl = pd.merge(df_ce_map, ce_weights, how = 'left', \
                         left_on = ['Week', 'Strike Price'], \
                         right_on = ['Date', 'Strike'])
    
    df_ce_pfl['Pfl_weights'].replace('', np.nan, inplace=True)
    df_ce_pfl.dropna(subset=['Pfl_weights'], inplace=True)
    
    df_ce_pfl = df_ce_pfl[['Week', 'Day', 'Expiry', 'Month', 'Strike Price', 'Close', 'Pfl_weights']]
    df_ce_pfl['hedge_value'] = df_ce_pfl['Close'] * df_ce_pfl['Pfl_weights'] 
    df_ce_pfl = df_ce_pfl.groupby(['Week', 'Day', 'Expiry', 'Month'])[['hedge_value']].sum().reset_index()
    df_ce_pfl = df_ce_pfl.assign(x=pd.to_datetime(df_ce_pfl['Day']))
    df_ce_pfl = df_ce_pfl.assign(y=pd.to_datetime(df_ce_pfl['Week']))
    df_ce_pfl = df_ce_pfl.sort_values(by =['y','x']).drop(['x', 'y'], 1).reset_index(drop=True)
    
    df_pe_pfl = pd.merge(df_pe_map, pe_weights, how = 'left', \
                         left_on = ['Week', 'Strike Price'], \
                         right_on = ['Date', 'Strike'])
    df_pe_pfl['Pfl_weights'].replace('', np.nan, inplace=True)
    df_pe_pfl.dropna(subset=['Pfl_weights'], inplace=True)
    
    df_pe_pfl = df_pe_pfl[['Week', 'Day', 'Expiry', 'Month', 'Strike Price', 'Close', 'Pfl_weights']]
    df_pe_pfl['hedge_value'] = df_pe_pfl['Close'] * df_pe_pfl['Pfl_weights']  
    df_pe_pfl = df_pe_pfl.groupby(['Week', 'Day', 'Expiry', 'Month'])[['hedge_value']].sum().reset_index()
    df_pe_pfl = df_pe_pfl.assign(x=pd.to_datetime(df_pe_pfl['Day']))
    df_pe_pfl = df_pe_pfl.assign(y=pd.to_datetime(df_pe_pfl['Week']))
    df_pe_pfl = df_pe_pfl.sort_values(by =['y','x']).drop(['x', 'y'], 1).reset_index(drop=True)
    

    df_cash_pfl = pd.merge(df_wk_day_map, cash_invst, how = 'left', \
                         left_on = ['Week'], \
                         right_on = ['Date'])
    df_cash_pfl = df_cash_pfl[['Week', 'Day', 'Expiry', 'Month', 'Pfl_weights']]

    # Time value is weekend to current date as cash is fixed at weekend
    df_cash_pfl['time_value'] = (pd.to_datetime(df_cash_pfl['Expiry'], format= "%d-%b-%Y") - pd.to_datetime(df_cash_pfl['Day'], format= "%d-%b-%Y")).dt.days / 365
    
    df_cash_pfl['r_f'] = df_cash_pfl.apply(lambda x: get_risk_free_rate_from_exact_date(datetime.strptime(x['Day'], "%d-%b-%Y").date()), axis=1) 

    # Cash is dicounted as cash is moedelled as of future time and to have that, we just need a discounted cash at prior time
    df_cash_pfl['Factor'] = df_cash_pfl.apply(lambda x: np.exp(- x['time_value'] * x['r_f']), axis=1)

    df_cash_pfl['hedge_value'] = df_cash_pfl['Factor'] * df_cash_pfl['Pfl_weights']
    
    df_static_hedge = df_ce_pfl.merge(df_pe_pfl , how = 'left', left_on=['Day', 'Expiry'], right_on=['Day', 'Expiry'],\
                                suffixes=('_CE', '_PE'))
    
    
    df_static_hedge = df_static_hedge.merge(df_cash_pfl,  how = 'left', left_on=['Day', 'Expiry'], right_on=['Day', 'Expiry'])

    if (code_id == "0A"):
        df_static_hedge['Hedge_Pfl_value'] = df_static_hedge['hedge_value_CE']
    else:
        df_static_hedge['Hedge_Pfl_value'] = df_static_hedge['hedge_value_CE'] + \
                                             df_static_hedge['hedge_value_PE'] + \
                                             df_static_hedge['hedge_value']
    
    df_static_hedge['Week'] = df_static_hedge['Week_CE']
    df_static_hedge = df_static_hedge.drop(columns=['Week_CE', 'Week_PE'])
                                     
    df_static_hedge = df_static_hedge[['Week', 'Day', 'Expiry', 'Month', 'Hedge_Pfl_value']]

    # Monthly Option Value is consistent with Settlemet Price - Changed in the function 'generate_dict_opt_value'
    df_static_hedge = pd.merge(df_static_hedge, df_monthly_option_value, how = 'left', left_on = ['Day', 'Month'], right_on = ['Date', 'Expiry'], suffixes=('', 'OPT'))

    df_static_hedge['Static Hedge Error'] = df_static_hedge['Close Price'] - df_static_hedge['Hedge_Pfl_value']
    
    df_static_hedge = df_static_hedge[['Month', 'Week', 'Day', 'Expiry', 'Close Price', 'Hedge_Pfl_value', 'Static Hedge Error']]
    df_static_hedge = df_static_hedge.rename(columns={"Month":"Month_Expiry", "Week":"Week_Start", "Expiry":"Week_Expiry", "Day":"Date"})

    df_static_hedge.to_csv(output_path + "C" + code_id + "_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + ".csv", index = False)
    
    return df_static_hedge



In [25]:

#Find the monthly strike of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)
dict_daily_dates_error_cal = generate_daily_dates_each_month(dict_wkly_expiries_each_month, holidays_list)

# #Load all monthly mkt data
mthly_mkt_data = load_all_mthly_data(mthly_expiries_list, input_data_path, holidays_list, prod_type_lists=["FUT", "CE", "PE"], stock_ident = stock_ident)

df_wk_day_map = create_df_wk_days(dict_daily_dates_error_cal, dict_wkly_expiries_each_month, holidays_list)

df_ce_weekly = combine_weekly_option_df(dict_wkly_expiries_each_month, prod_type="CE", 
                                        input_path=input_data_path, stock_ident=stock_ident)

df_ce_weekly['Date'] = df_ce_weekly.apply(lambda x: x['Date'].split("-")[0] + "-" + \
                                  x['Date'].split("-")[1]  + "-"  + "20" + x['Date'].split("-")[2] \
                                  if (len(x['Date'].split("-")[2]) == 2) else x['Date'], axis=1) 

df_ce_weekly['Expiry'] = df_ce_weekly.apply(lambda x: x['Expiry'].split("-")[0] + "-" + \
                                  x['Expiry'].split("-")[1]  + "-"  + "20" + x['Expiry'].split("-")[2] \
                                  if (len(x['Expiry'].split("-")[2]) == 2) else x['Expiry'], axis=1) 

df_pe_weekly = combine_weekly_option_df(dict_wkly_expiries_each_month, prod_type="PE", 
                                        input_path=input_data_path, stock_ident=stock_ident)


df_pe_weekly['Date'] = df_pe_weekly.apply(lambda x: x['Date'].split("-")[0] + "-" + \
                                  x['Date'].split("-")[1]  + "-"  + "20" + x['Date'].split("-")[2] \
                                  if (len(x['Date'].split("-")[2]) == 2) else x['Date'], axis=1) 

df_pe_weekly['Expiry'] = df_pe_weekly.apply(lambda x: x['Expiry'].split("-")[0] + "-" + \
                                  x['Expiry'].split("-")[1]  + "-"  + "20" + x['Expiry'].split("-")[2] \
                                  if (len(x['Expiry'].split("-")[2]) == 2) else x['Expiry'], axis=1) 


if (prod_type == "CE"):
#     code_id_list = ["0B", "11", "11A", "12", "12A", "13", "13A", "14", "14A"]
    code_id_list = ["0A", "6A", "7A", "8A", "9A"]
else:
#     code_id_list = ["11", "11A", "12", "12A", "13", "13A", "14", "14A"]
    code_id_list = ["6A", "7A", "8A", "9A"]
    
for code_id in code_id_list:

    df_monthly_option_value = generate_dict_opt_value(dict_daily_dates_error_cal,mthly_mkt_data, holidays_list, \
                                                      prod_type=prod_type, prod_moneyness=prod_moneyness, output_path=output_data_path, stock_ident=stock_ident, code_id=code_id)
    df_monthly_option_value['Date'] = df_monthly_option_value.apply(lambda x: x['Date'].split("-")[0] + "-" + \
                                  x['Date'].split("-")[1]  + "-"  + "20" + x['Date'].split("-")[2] \
                                  if (len(x['Date'].split("-")[2]) == 2) else x['Date'], axis=1) 
    df_monthly_option_value['Expiry'] = df_monthly_option_value.apply(lambda x: x['Expiry'].split("-")[0] + "-" + \
                                  x['Expiry'].split("-")[1]  + "-"  + "20" + x['Expiry'].split("-")[2] \
                                  if (len(x['Expiry'].split("-")[2]) == 2) else x['Expiry'], axis=1)


    df_static_hedge = find_daily_static_hedging_error(df_wk_day_map, df_ce_weekly, df_pe_weekly, df_monthly_option_value, \
                                                      prod_type=prod_type, prod_moneyness=prod_moneyness, holidays_list=holidays_list, \
                                                      output_path=output_data_path, code_id=code_id)


#Final output
#DataFrame: "Date", "Option Value", "Quantity" - this to be considered later, "Static Hedging Error", "Dynamic Hedging Error"


/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand